<a href="https://colab.research.google.com/github/aaptecode/DS-Unit-4-Sprint-3-Neural-Networks/blob/master/LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


## Importing Basic Packages

In [1]:
!pip install category_encoders


    100% |████████████████████████████████| 61kB 2.4MB/s 


In [0]:
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
np.random.seed(237)
style.use('seaborn-whitegrid')


## Data Wrangling

In [3]:
##### Your Code Here #####

url = 'https://raw.githubusercontent.com/aaptecode/DS-Unit-4-Sprint-3-Neural-Networks/master/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df = pd.read_csv(url)
print(df.shape, "\n")
df.head()

(7043, 21) 



,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.TotalCharges.replace(' ', '0.0', inplace=True)
df.TotalCharges = df.TotalCharges.astype(float)
df.drop('customerID', axis='columns', inplace=True)

df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
y = df['Churn']
X = df.drop('Churn', axis='columns')
X.shape, y.shape

((7043, 19), (7043,))

## Tuning the Hyperparameters

Using GridSearchCV and tuning one variable at a time to save time. 

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor

import category_encoders as ce
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import RobustScaler



Using TensorFlow backend.


### Batch Size

In [7]:
%%time

# Function to create model, required for KerasClassifier
def create_model(activation='sigmoid', xtra_layers = 1):
    # create model
    print(f'Using activation {activation}')
    model = Sequential()
    model.add(Dense(12, input_dim=44, activation=activation))
    for i in range(xtra_layers):
        model.add(Dense(12, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create pipeline
pipeline = make_pipeline(\
                         ce.BinaryEncoder(),
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [30, 40, 50],
    'kerasclassifier__activation': ['tanh'],
    'kerasclassifier__epochs': [25]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=5, 
                         scoring='accuracy', verbose=10)
gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('\n\nCross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)
print()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] kerasclassifier__activation=tanh, kerasclassifier__batch_size=30, kerasclassifier__epochs=25 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Using activation tanh
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/25
5634/5634 [==============================] - 1s 180us/step - loss: 0.4770 - acc: 0.7648
Epoch 2/25
5634/5634 [==============================] - 0s 48us/step - loss: 0.4328 - acc: 0.7994
Epoch 3/25
5634/5634 [==============================] - 0s 47us/step - loss: 0.4271 - acc: 0.8000
Epoch 4/25
5634/5634 [==============================] - 0s 48us/step - loss: 0.4256 - acc: 0.8012
Epoch 5/25
5634/5634 [==============================] - 0s 47us/step - loss: 0.4245 - acc: 0.8010
Epoch 6/25
5634/5634 [==============================] - 0s 45us/step - loss: 0.4221 - acc: 0.8009
Epoch 7/25
5634/5634 [==============================] - 0s 45us/step - loss: 0.4209 - acc: 0.8030
Epoch 8/25
5634/5634 [==============================] - 0s 50us/step - loss: 0.4197 - acc: 0.8035
Epoch 9/25
5634/5634 [==============================] - 0s 47us/step - lo

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.8s remaining:    0.0s


Epoch 1/25
5634/5634 [==============================] - 0s 84us/step - loss: 0.5012 - acc: 0.7577
Epoch 2/25
5634/5634 [==============================] - 0s 40us/step - loss: 0.4376 - acc: 0.7973
Epoch 3/25
5634/5634 [==============================] - 0s 41us/step - loss: 0.4293 - acc: 0.7993
Epoch 4/25
5634/5634 [==============================] - 0s 40us/step - loss: 0.4264 - acc: 0.8023
Epoch 5/25
5634/5634 [==============================] - 0s 42us/step - loss: 0.4244 - acc: 0.7991
Epoch 6/25
5634/5634 [==============================] - 0s 41us/step - loss: 0.4211 - acc: 0.8021
Epoch 7/25
5634/5634 [==============================] - 0s 41us/step - loss: 0.4208 - acc: 0.8000
Epoch 8/25
5634/5634 [==============================] - 0s 41us/step - loss: 0.4196 - acc: 0.8019
Epoch 9/25
5634/5634 [==============================] - 0s 41us/step - loss: 0.4186 - acc: 0.8032
Epoch 10/25
5634/5634 [==============================] - 0s 42us/step - loss: 0.4181 - acc: 0.8026
Epoch 11/25
5634/56

...

Epoch 1/25
5635/5635 [==============================] - 1s 164us/step - loss: 0.5343 - acc: 0.7184
Epoch 2/25
5635/5635 [==============================] - 0s 33us/step - loss: 0.4401 - acc: 0.7959
Epoch 3/25
5635/5635 [==============================] - 0s 33us/step - loss: 0.4261 - acc: 0.8025
Epoch 4/25
5635/5635 [==============================] - 0s 34us/step - loss: 0.4206 - acc: 0.8020
Epoch 5/25
5635/5635 [==============================] - 0s 35us/step - loss: 0.4169 - acc: 0.8021
Epoch 6/25
5635/5635 [==============================] - 0s 35us/step - loss: 0.4148 - acc: 0.8046
Epoch 7/25
5635/5635 [==============================] - 0s 35us/step - loss: 0.4127 - acc: 0.8053
Epoch 8/25
5635/5635 [==============================] - 0s 37us/step - loss: 0.4104 - acc: 0.8059
Epoch 9/25
5635/5635 [==============================] - 0s 34us/step - loss: 0.4100 - acc: 0.8034
Epoch 10/25
5635/5635 [==============================] - 0s 35us/step - loss: 0.4088 - acc: 0.8066
Epoch 11/25
5635/5

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.1min remaining:    0.0s


Epoch 1/25
5635/5635 [==============================] - 1s 182us/step - loss: 0.5079 - acc: 0.7517
Epoch 2/25
5635/5635 [==============================] - 0s 33us/step - loss: 0.4308 - acc: 0.7957
Epoch 3/25
5635/5635 [==============================] - 0s 35us/step - loss: 0.4227 - acc: 0.7998
Epoch 4/25
5635/5635 [==============================] - 0s 37us/step - loss: 0.4192 - acc: 0.8048
Epoch 5/25
5635/5635 [==============================] - 0s 36us/step - loss: 0.4167 - acc: 0.8039
Epoch 6/25
5635/5635 [==============================] - 0s 36us/step - loss: 0.4143 - acc: 0.8073
Epoch 7/25
5635/5635 [==============================] - 0s 35us/step - loss: 0.4150 - acc: 0.8050
Epoch 8/25
5635/5635 [==============================] - 0s 35us/step - loss: 0.4137 - acc: 0.8046
Epoch 9/25
5635/5635 [==============================] - 0s 37us/step - loss: 0.4123 - acc: 0.8062
Epoch 10/25
5635/5635 [==============================] - 0s 36us/step - loss: 0.4117 - acc: 0.8048
Epoch 11/25
5635/5

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.8min finished


Using activation tanh
Epoch 1/25
7043/7043 [==============================] - 2s 217us/step - loss: 0.4795 - acc: 0.7698
Epoch 2/25
7043/7043 [==============================] - 0s 39us/step - loss: 0.4289 - acc: 0.7980
Epoch 3/25
7043/7043 [==============================] - 0s 38us/step - loss: 0.4239 - acc: 0.8012
Epoch 4/25
7043/7043 [==============================] - 0s 42us/step - loss: 0.4210 - acc: 0.8025
Epoch 5/25
7043/7043 [==============================] - 0s 38us/step - loss: 0.4180 - acc: 0.8046
Epoch 6/25
7043/7043 [==============================] - 0s 38us/step - loss: 0.4165 - acc: 0.8048
Epoch 7/25
7043/7043 [==============================] - 0s 38us/step - loss: 0.4149 - acc: 0.8052
Epoch 8/25
7043/7043 [==============================] - 0s 40us/step - loss: 0.4140 - acc: 0.8042
Epoch 9/25
7043/7043 [==============================] - 0s 39us/step - loss: 0.4129 - acc: 0.8063
Epoch 10/25
7043/7043 [==============================] - 0s 38us/step - loss: 0.4123 - acc: 0.8

**Batch size = 40**

### Activation Method

In [10]:
%%time

# Function to create model, required for KerasClassifier
def create_model(activation='sigmoid', xtra_layers = 1):
    # create model
    print(f'Using activation {activation}')
    model = Sequential()
    model.add(Dense(12, input_dim=44, activation=activation))
    for i in range(xtra_layers):
        model.add(Dense(12, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create pipeline
pipeline = make_pipeline(\
                         ce.BinaryEncoder(),
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [40],
    'kerasclassifier__activation': ['tanh', 'relu', 'sigmoid', 'softmax', 'elu', 'selu'],
    'kerasclassifier__epochs': [25]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=5, 
                         scoring='accuracy', verbose=10)
gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] kerasclassifier__activation=tanh, kerasclassifier__batch_size=40, kerasclassifier__epochs=25 
Using activation tanh


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/25
5634/5634 [==============================] - 6s 997us/step - loss: 0.5265 - acc: 0.7441
Epoch 2/25
5634/5634 [==============================] - 0s 83us/step - loss: 0.4416 - acc: 0.7916
Epoch 3/25
5634/5634 [==============================] - 0s 80us/step - loss: 0.4304 - acc: 0.7977
Epoch 4/25
5634/5634 [==============================] - 0s 83us/step - loss: 0.4261 - acc: 0.8019
Epoch 5/25
5634/5634 [==============================] - 0s 69us/step - loss: 0.4239 - acc: 0.8035
Epoch 6/25
5634/5634 [==============================] - 0s 65us/step - loss: 0.4221 - acc: 0.8060
Epoch 7/25
5634/5634 [==============================] - 0s 64us/step - loss: 0.4206 - acc: 0.8060
Epoch 8/25
5634/5634 [==============================] - 0s 65us/step - loss: 0.4198 - acc: 0.8051
Epoch 9/25
5634/5634 [==============================] - 0s 69us/step - loss: 0.4191 - acc: 0.8056
Epoch 10/25
5634/5634 [==============================] - 0s 67us/step - loss: 0.4180 - acc: 0.8058
Epoch 11/25
5634/5

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.1s remaining:    0.0s


Epoch 1/25
5634/5634 [==============================] - 5s 925us/step - loss: 0.5359 - acc: 0.7259
Epoch 2/25
5634/5634 [==============================] - 0s 67us/step - loss: 0.4513 - acc: 0.7909
Epoch 3/25
5634/5634 [==============================] - 0s 68us/step - loss: 0.4382 - acc: 0.7964
Epoch 4/25
5634/5634 [==============================] - 0s 64us/step - loss: 0.4322 - acc: 0.7978
Epoch 5/25
5634/5634 [==============================] - 0s 67us/step - loss: 0.4294 - acc: 0.8000
Epoch 6/25
5634/5634 [==============================] - 0s 66us/step - loss: 0.4271 - acc: 0.8012
Epoch 7/25
5634/5634 [==============================] - 0s 64us/step - loss: 0.4252 - acc: 0.8032
Epoch 8/25
5634/5634 [==============================] - 0s 66us/step - loss: 0.4235 - acc: 0.8009
Epoch 9/25
5634/5634 [==============================] - 0s 63us/step - loss: 0.4222 - acc: 0.8023
Epoch 10/25
5634/5634 [==============================] - 0s 64us/step - loss: 0.4203 - acc: 0.8028
Epoch 11/25
5634/5

...

Epoch 1/25
5635/5635 [==============================] - 6s 1ms/step - loss: 0.5708 - acc: 0.6841
Epoch 2/25
5635/5635 [==============================] - 0s 76us/step - loss: 0.4543 - acc: 0.7659
Epoch 3/25
5635/5635 [==============================] - 0s 76us/step - loss: 0.4298 - acc: 0.7941
Epoch 4/25
5635/5635 [==============================] - 0s 73us/step - loss: 0.4240 - acc: 0.8004
Epoch 5/25
5635/5635 [==============================] - 0s 77us/step - loss: 0.4193 - acc: 0.8025
Epoch 6/25
5635/5635 [==============================] - 0s 72us/step - loss: 0.4155 - acc: 0.8055
Epoch 7/25
5635/5635 [==============================] - 0s 71us/step - loss: 0.4135 - acc: 0.8069
Epoch 8/25
5635/5635 [==============================] - 0s 69us/step - loss: 0.4108 - acc: 0.8099
Epoch 9/25
5635/5635 [==============================] - 0s 72us/step - loss: 0.4093 - acc: 0.8087
Epoch 10/25
5635/5635 [==============================] - 0s 72us/step - loss: 0.4077 - acc: 0.8083
Epoch 11/25
5635/563

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.8min remaining:    0.0s


Epoch 1/25
5635/5635 [==============================] - 6s 1ms/step - loss: 0.4930 - acc: 0.7673
Epoch 2/25
5635/5635 [==============================] - 0s 73us/step - loss: 0.4317 - acc: 0.7975
Epoch 3/25
5635/5635 [==============================] - 0s 73us/step - loss: 0.4224 - acc: 0.8046
Epoch 4/25
5635/5635 [==============================] - 0s 73us/step - loss: 0.4175 - acc: 0.8037
Epoch 5/25
5635/5635 [==============================] - 0s 71us/step - loss: 0.4143 - acc: 0.8060
Epoch 6/25
5635/5635 [==============================] - 0s 75us/step - loss: 0.4132 - acc: 0.8067
Epoch 7/25
5635/5635 [==============================] - 0s 73us/step - loss: 0.4114 - acc: 0.8064
Epoch 8/25
5635/5635 [==============================] - 0s 73us/step - loss: 0.4097 - acc: 0.8087
Epoch 9/25
5635/5635 [==============================] - 0s 75us/step - loss: 0.4096 - acc: 0.8055
Epoch 10/25
5635/5635 [==============================] - 0s 76us/step - loss: 0.4094 - acc: 0.8076
Epoch 11/25
5635/563

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 10.4min finished



Epoch 1/25
7043/7043 [==============================] - 8s 1ms/step - loss: 0.5394 - acc: 0.7125
Epoch 2/25
7043/7043 [==============================] - 1s 84us/step - loss: 0.4395 - acc: 0.7930
Epoch 3/25
7043/7043 [==============================] - 1s 87us/step - loss: 0.4279 - acc: 0.7978
Epoch 4/25
7043/7043 [==============================] - 1s 81us/step - loss: 0.4230 - acc: 0.7995
Epoch 5/25
7043/7043 [==============================] - 1s 81us/step - loss: 0.4189 - acc: 0.8034
Epoch 6/25
7043/7043 [==============================] - 1s 83us/step - loss: 0.4172 - acc: 0.8039
Epoch 7/25
7043/7043 [==============================] - 1s 80us/step - loss: 0.4157 - acc: 0.8055
Epoch 8/25
7043/7043 [==============================] - 1s 81us/step - loss: 0.4143 - acc: 0.8041
Epoch 9/25
7043/7043 [==============================] - 1s 80us/step - loss: 0.4136 - acc: 0.8052
Epoch 10/25
7043/7043 [==============================] - 1s 81us/step - loss: 0.4122 - acc: 0.8059
Epoch 11/25
7043/70

**Activation = tanh**

### Number of Extra Layers

In [11]:
%%time

# Function to create model, required for KerasClassifier
def create_model(activation='sigmoid', xtra_layers = 1):
    # create model
    print(f'Using activation {activation}')
    model = Sequential()
    model.add(Dense(12, input_dim=44, activation=activation))
    for i in range(xtra_layers):
        model.add(Dense(12, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create pipeline
pipeline = make_pipeline(\
                         ce.BinaryEncoder(),
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [40],
    'kerasclassifier__activation': ['tanh'],
    'kerasclassifier__xtra_layers': [0,1,2],
    'kerasclassifier__epochs': [25]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=5, 
                         scoring='accuracy', verbose=10)
gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] kerasclassifier__activation=tanh, kerasclassifier__batch_size=40, kerasclassifier__epochs=25, kerasclassifier__xtra_layers=0 
Using activation tanh


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/25
5634/5634 [==============================] - 8s 1ms/step - loss: 0.5835 - acc: 0.6704
Epoch 2/25
5634/5634 [==============================] - 0s 83us/step - loss: 0.4554 - acc: 0.7863
Epoch 3/25
5634/5634 [==============================] - 0s 86us/step - loss: 0.4349 - acc: 0.7952
Epoch 4/25
5634/5634 [==============================] - 0s 83us/step - loss: 0.4291 - acc: 0.7987
Epoch 5/25
5634/5634 [==============================] - 0s 82us/step - loss: 0.4254 - acc: 0.8000
Epoch 6/25
5634/5634 [==============================] - 0s 83us/step - loss: 0.4232 - acc: 0.8012
Epoch 7/25
5634/5634 [==============================] - 0s 85us/step - loss: 0.4218 - acc: 0.8023
Epoch 8/25
5634/5634 [==============================] - 0s 84us/step - loss: 0.4212 - acc: 0.8055
Epoch 9/25
5634/5634 [==============================] - 0s 81us/step - loss: 0.4200 - acc: 0.8055
Epoch 10/25
5634/5634 [==============================] - 0s 83us/step - loss: 0.4194 - acc: 0.8055
Epoch 11/25
5634/563

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.5s remaining:    0.0s


Epoch 1/25
5634/5634 [==============================] - 8s 1ms/step - loss: 0.5247 - acc: 0.7414
Epoch 2/25
5634/5634 [==============================] - 0s 86us/step - loss: 0.4464 - acc: 0.7898
Epoch 3/25
5634/5634 [==============================] - 0s 86us/step - loss: 0.4339 - acc: 0.7938
Epoch 4/25
5634/5634 [==============================] - 0s 85us/step - loss: 0.4294 - acc: 0.7985
Epoch 5/25
5634/5634 [==============================] - 0s 86us/step - loss: 0.4266 - acc: 0.7975
Epoch 6/25
5634/5634 [==============================] - 1s 91us/step - loss: 0.4250 - acc: 0.8021
Epoch 7/25
5634/5634 [==============================] - 0s 89us/step - loss: 0.4238 - acc: 0.8019
Epoch 8/25
5634/5634 [==============================] - 0s 87us/step - loss: 0.4230 - acc: 0.8032
Epoch 9/25
5634/5634 [==============================] - 0s 81us/step - loss: 0.4216 - acc: 0.8017
Epoch 10/25
5634/5634 [==============================] - 0s 81us/step - loss: 0.4209 - acc: 0.8035
Epoch 11/25
5634/563

...

Epoch 1/25
5635/5635 [==============================] - 8s 1ms/step - loss: 0.5169 - acc: 0.7414
Epoch 2/25
5635/5635 [==============================] - 1s 92us/step - loss: 0.4365 - acc: 0.7924
Epoch 3/25
5635/5635 [==============================] - 1s 91us/step - loss: 0.4249 - acc: 0.7965
Epoch 4/25
5635/5635 [==============================] - 1s 92us/step - loss: 0.4198 - acc: 0.8014
Epoch 5/25
5635/5635 [==============================] - 1s 90us/step - loss: 0.4173 - acc: 0.8030
Epoch 6/25
5635/5635 [==============================] - 1s 95us/step - loss: 0.4150 - acc: 0.8046
Epoch 7/25
5635/5635 [==============================] - 1s 97us/step - loss: 0.4137 - acc: 0.8034
Epoch 8/25
5635/5635 [==============================] - 1s 95us/step - loss: 0.4120 - acc: 0.8016
Epoch 9/25
5635/5635 [==============================] - 1s 92us/step - loss: 0.4130 - acc: 0.8039
Epoch 10/25
5635/5635 [==============================] - 1s 92us/step - loss: 0.4099 - acc: 0.8041
Epoch 11/25
5635/563

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  3.7min remaining:    0.0s


Epoch 1/25
5635/5635 [==============================] - 8s 1ms/step - loss: 0.5118 - acc: 0.7571
Epoch 2/25
5635/5635 [==============================] - 0s 88us/step - loss: 0.4329 - acc: 0.7934
Epoch 3/25
5635/5635 [==============================] - 1s 95us/step - loss: 0.4250 - acc: 0.8004
Epoch 4/25
5635/5635 [==============================] - 1s 91us/step - loss: 0.4207 - acc: 0.8035
Epoch 5/25
5635/5635 [==============================] - 1s 90us/step - loss: 0.4183 - acc: 0.8027
Epoch 6/25
5635/5635 [==============================] - 1s 94us/step - loss: 0.4166 - acc: 0.8025
Epoch 7/25
5635/5635 [==============================] - 1s 91us/step - loss: 0.4161 - acc: 0.8039
Epoch 8/25
5635/5635 [==============================] - 1s 91us/step - loss: 0.4138 - acc: 0.8060
Epoch 9/25
5635/5635 [==============================] - 0s 87us/step - loss: 0.4125 - acc: 0.8059
Epoch 10/25
5635/5635 [==============================] - 0s 87us/step - loss: 0.4123 - acc: 0.8043
Epoch 11/25
5635/563

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  6.4min finished


Using activation tanh
Epoch 1/25
7043/7043 [==============================] - 9s 1ms/step - loss: 0.5105 - acc: 0.7449
Epoch 2/25
7043/7043 [==============================] - 1s 101us/step - loss: 0.4412 - acc: 0.7900
Epoch 3/25
7043/7043 [==============================] - 1s 98us/step - loss: 0.4299 - acc: 0.7974
Epoch 4/25
7043/7043 [==============================] - 1s 93us/step - loss: 0.4256 - acc: 0.7998
Epoch 5/25
7043/7043 [==============================] - 1s 93us/step - loss: 0.4228 - acc: 0.8001
Epoch 6/25
7043/7043 [==============================] - 1s 97us/step - loss: 0.4212 - acc: 0.8011
Epoch 7/25
7043/7043 [==============================] - 1s 97us/step - loss: 0.4192 - acc: 0.8042
Epoch 8/25
7043/7043 [==============================] - 1s 96us/step - loss: 0.4177 - acc: 0.8043
Epoch 9/25
7043/7043 [==============================] - 1s 92us/step - loss: 0.4170 - acc: 0.8032
Epoch 10/25
7043/7043 [==============================] - 1s 93us/step - loss: 0.4158 - acc: 0.80

**Number of Extra Layers = 0**

### Number of cells per hidden layer

In [12]:
%%time

# Function to create model, required for KerasClassifier
def create_model(activation='sigmoid', xtra_layers = 0, cell_no = 12):
    # create model
    print(f'Using activation {activation}')
    model = Sequential()
    model.add(Dense(cell_no, input_dim=44, activation=activation))
    for i in range(xtra_layers):
        model.add(Dense(cell_no, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create pipeline
pipeline = make_pipeline(\
                         ce.BinaryEncoder(),
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [40],
    'kerasclassifier__activation': ['tanh'],
    'kerasclassifier__xtra_layers': [0],
    'kerasclassifier__cell_no': [12, 18, 24, 30],
    'kerasclassifier__epochs': [25]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=5, 
                         scoring='accuracy', verbose=10)
gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] kerasclassifier__activation=tanh, kerasclassifier__batch_size=40, kerasclassifier__cell_no=12, kerasclassifier__epochs=25, kerasclassifier__xtra_layers=0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Using activation tanh
Epoch 1/25
5634/5634 [==============================] - 10s 2ms/step - loss: 0.5610 - acc: 0.7387
Epoch 2/25
5634/5634 [==============================] - 1s 109us/step - loss: 0.4457 - acc: 0.7882
Epoch 3/25
5634/5634 [==============================] - 1s 117us/step - loss: 0.4315 - acc: 0.7969
Epoch 4/25
5634/5634 [==============================] - 1s 104us/step - loss: 0.4267 - acc: 0.8023
Epoch 5/25
5634/5634 [==============================] - 1s 99us/step - loss: 0.4245 - acc: 0.8021
Epoch 6/25
5634/5634 [==============================] - 1s 91us/step - loss: 0.4231 - acc: 0.8048
Epoch 7/25
5634/5634 [==============================] - 1s 91us/step - loss: 0.4217 - acc: 0.8033
Epoch 8/25
5634/5634 [==============================] - 1s 99us/step - loss: 0.4208 - acc: 0.8049
Epoch 9/25
5634/5634 [==============================] - 1s 98us/step - loss: 0.4206 - acc: 0.8042
Epoch 10/25
5634/5634 [==============================] - 1s 100us/step - loss: 0.4193 - acc: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.0s remaining:    0.0s


Epoch 1/25
5634/5634 [==============================] - 9s 2ms/step - loss: 0.6000 - acc: 0.6640
Epoch 2/25
5634/5634 [==============================] - 1s 90us/step - loss: 0.4509 - acc: 0.7882
Epoch 3/25
5634/5634 [==============================] - 1s 90us/step - loss: 0.4334 - acc: 0.7948
Epoch 4/25
5634/5634 [==============================] - 1s 95us/step - loss: 0.4288 - acc: 0.7966
Epoch 5/25
5634/5634 [==============================] - 1s 100us/step - loss: 0.4261 - acc: 0.7998
Epoch 6/25
5634/5634 [==============================] - 1s 96us/step - loss: 0.4245 - acc: 0.8007
Epoch 7/25
5634/5634 [==============================] - 1s 98us/step - loss: 0.4232 - acc: 0.8017
Epoch 8/25
5634/5634 [==============================] - 1s 105us/step - loss: 0.4224 - acc: 0.8024
Epoch 9/25
5634/5634 [==============================] - 1s 100us/step - loss: 0.4209 - acc: 0.8039
Epoch 10/25
5634/5634 [==============================] - 1s 93us/step - loss: 0.4207 - acc: 0.8016
Epoch 11/25
5634/

...

Epoch 1/25
5635/5635 [==============================] - 10s 2ms/step - loss: 0.4849 - acc: 0.7606
Epoch 2/25
5635/5635 [==============================] - 1s 101us/step - loss: 0.4340 - acc: 0.7902
Epoch 3/25
5635/5635 [==============================] - 1s 99us/step - loss: 0.4237 - acc: 0.7975
Epoch 4/25
5635/5635 [==============================] - 1s 97us/step - loss: 0.4197 - acc: 0.7986
Epoch 5/25
5635/5635 [==============================] - 1s 106us/step - loss: 0.4167 - acc: 0.8021
Epoch 6/25
5635/5635 [==============================] - 1s 108us/step - loss: 0.4147 - acc: 0.7991
Epoch 7/25
5635/5635 [==============================] - 1s 103us/step - loss: 0.4140 - acc: 0.7996
Epoch 8/25
5635/5635 [==============================] - 1s 105us/step - loss: 0.4118 - acc: 0.8032
Epoch 9/25
5635/5635 [==============================] - 1s 102us/step - loss: 0.4110 - acc: 0.8039
Epoch 10/25
5635/5635 [==============================] - 1s 100us/step - loss: 0.4104 - acc: 0.8027
Epoch 11/25


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  4.3min remaining:    0.0s


Epoch 1/25
5635/5635 [==============================] - 10s 2ms/step - loss: 0.5159 - acc: 0.7530
Epoch 2/25
5635/5635 [==============================] - 1s 108us/step - loss: 0.4367 - acc: 0.7931
Epoch 3/25
5635/5635 [==============================] - 1s 104us/step - loss: 0.4260 - acc: 0.7998
Epoch 4/25
5635/5635 [==============================] - 1s 100us/step - loss: 0.4219 - acc: 0.8053
Epoch 5/25
5635/5635 [==============================] - 1s 95us/step - loss: 0.4196 - acc: 0.8046
Epoch 6/25
5635/5635 [==============================] - 1s 100us/step - loss: 0.4172 - acc: 0.8014
Epoch 7/25
5635/5635 [==============================] - 1s 96us/step - loss: 0.4166 - acc: 0.8055
Epoch 8/25
5635/5635 [==============================] - 1s 95us/step - loss: 0.4155 - acc: 0.8066
Epoch 9/25
5635/5635 [==============================] - 1s 98us/step - loss: 0.4143 - acc: 0.8048
Epoch 10/25
5635/5635 [==============================] - 1s 92us/step - loss: 0.4134 - acc: 0.8062
Epoch 11/25
563

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  9.8min finished


Using activation tanh
Epoch 1/25
7043/7043 [==============================] - 11s 2ms/step - loss: 0.5239 - acc: 0.7304
Epoch 2/25
7043/7043 [==============================] - 1s 100us/step - loss: 0.4288 - acc: 0.7968
Epoch 3/25
7043/7043 [==============================] - 1s 101us/step - loss: 0.4230 - acc: 0.7982
Epoch 4/25
7043/7043 [==============================] - 1s 102us/step - loss: 0.4206 - acc: 0.8001
Epoch 5/25
7043/7043 [==============================] - 1s 100us/step - loss: 0.4186 - acc: 0.8001
Epoch 6/25
7043/7043 [==============================] - 1s 102us/step - loss: 0.4172 - acc: 0.8021
Epoch 7/25
7043/7043 [==============================] - 1s 105us/step - loss: 0.4159 - acc: 0.8035
Epoch 8/25
7043/7043 [==============================] - 1s 108us/step - loss: 0.4144 - acc: 0.8041
Epoch 9/25
7043/7043 [==============================] - 1s 120us/step - loss: 0.4147 - acc: 0.8034
Epoch 10/25
7043/7043 [==============================] - 1s 103us/step - loss: 0.4141 - 

**Number of cells per hidden layer = 18**

### Epochs

In [13]:
%%time

# Function to create model, required for KerasClassifier
def create_model(activation='sigmoid', xtra_layers = 0, cell_no = 12):
    # create model
    print(f'Using activation {activation}')
    model = Sequential()
    model.add(Dense(cell_no, input_dim=44, activation=activation))
    for i in range(xtra_layers):
        model.add(Dense(cell_no, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create pipeline
pipeline = make_pipeline(\
                         ce.BinaryEncoder(),
                         RobustScaler(), 
                         KerasClassifier(build_fn=create_model, verbose=1))

# Model validation.
param_grid = {
    'kerasclassifier__batch_size': [40],
    'kerasclassifier__activation': ['tanh'],
    'kerasclassifier__xtra_layers': [0],
    'kerasclassifier__cell_no': [18],
    'kerasclassifier__epochs': [25, 50, 100, 150]
}

gridsearch = GridSearchCV(pipeline, param_grid=param_grid, cv=5, 
                         scoring='accuracy', verbose=10)
gridsearch.fit(X, y)

# Interpret the results.

# Best cross validation score
print('Cross Validation Score:', gridsearch.best_score_)

# Best parameters which resulted in the best score
print('Best Parameters:', gridsearch.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] kerasclassifier__activation=tanh, kerasclassifier__batch_size=40, kerasclassifier__cell_no=18, kerasclassifier__epochs=25, kerasclassifier__xtra_layers=0 
Using activation tanh


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/25
5634/5634 [==============================] - 11s 2ms/step - loss: 0.5264 - acc: 0.7231
Epoch 2/25
5634/5634 [==============================] - 1s 178us/step - loss: 0.4339 - acc: 0.7907
Epoch 3/25
5634/5634 [==============================] - 1s 112us/step - loss: 0.4247 - acc: 0.7973
Epoch 4/25
5634/5634 [==============================] - 1s 109us/step - loss: 0.4226 - acc: 0.8049
Epoch 5/25
5634/5634 [==============================] - 1s 109us/step - loss: 0.4214 - acc: 0.8021
Epoch 6/25
5634/5634 [==============================] - 1s 110us/step - loss: 0.4212 - acc: 0.8010
Epoch 7/25
5634/5634 [==============================] - 1s 106us/step - loss: 0.4199 - acc: 0.8033
Epoch 8/25
5634/5634 [==============================] - 1s 107us/step - loss: 0.4194 - acc: 0.8055
Epoch 9/25
5634/5634 [==============================] - 1s 100us/step - loss: 0.4191 - acc: 0.8044
Epoch 10/25
5634/5634 [==============================] - 1s 104us/step - loss: 0.4182 - acc: 0.8072
Epoch 11/2

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.8s remaining:    0.0s


Epoch 1/25
5634/5634 [==============================] - 11s 2ms/step - loss: 0.5078 - acc: 0.7407
Epoch 2/25
5634/5634 [==============================] - 1s 100us/step - loss: 0.4354 - acc: 0.7943
Epoch 3/25
5634/5634 [==============================] - 1s 101us/step - loss: 0.4278 - acc: 0.7993
Epoch 4/25
5634/5634 [==============================] - 1s 101us/step - loss: 0.4250 - acc: 0.8007
Epoch 5/25
5634/5634 [==============================] - 1s 104us/step - loss: 0.4230 - acc: 0.8016
Epoch 6/25
5634/5634 [==============================] - 1s 105us/step - loss: 0.4216 - acc: 0.8019
Epoch 7/25
5634/5634 [==============================] - 1s 101us/step - loss: 0.4209 - acc: 0.8012
Epoch 8/25
5634/5634 [==============================] - 1s 107us/step - loss: 0.4200 - acc: 0.8032
Epoch 9/25
5634/5634 [==============================] - 1s 105us/step - loss: 0.4194 - acc: 0.8046
Epoch 10/25
5634/5634 [==============================] - 1s 101us/step - loss: 0.4181 - acc: 0.8033
Epoch 11/2

...

Epoch 1/50
5635/5635 [==============================] - 12s 2ms/step - loss: 0.4740 - acc: 0.7727
Epoch 2/50
5635/5635 [==============================] - 1s 107us/step - loss: 0.4298 - acc: 0.7972
Epoch 3/50
5635/5635 [==============================] - 1s 105us/step - loss: 0.4225 - acc: 0.8000
Epoch 4/50
5635/5635 [==============================] - 1s 105us/step - loss: 0.4192 - acc: 0.8030
Epoch 5/50
5635/5635 [==============================] - 1s 105us/step - loss: 0.4160 - acc: 0.8044
Epoch 6/50
5635/5635 [==============================] - 1s 103us/step - loss: 0.4142 - acc: 0.8039
Epoch 7/50
5635/5635 [==============================] - 1s 105us/step - loss: 0.4135 - acc: 0.8051
Epoch 8/50
5635/5635 [==============================] - 1s 111us/step - loss: 0.4114 - acc: 0.8034
Epoch 9/50
5635/5635 [==============================] - 1s 108us/step - loss: 0.4107 - acc: 0.8025
Epoch 10/50
5635/5635 [==============================] - 1s 108us/step - loss: 0.4092 - acc: 0.8091
Epoch 11/5

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  5.8min remaining:    0.0s


Epoch 1/50
5635/5635 [==============================] - 12s 2ms/step - loss: 0.5149 - acc: 0.7429
Epoch 2/50
5635/5635 [==============================] - 1s 115us/step - loss: 0.4368 - acc: 0.7927
Epoch 3/50
5635/5635 [==============================] - 1s 109us/step - loss: 0.4269 - acc: 0.7980
Epoch 4/50
5635/5635 [==============================] - 1s 123us/step - loss: 0.4223 - acc: 0.8002
Epoch 5/50
5635/5635 [==============================] - 1s 107us/step - loss: 0.4195 - acc: 0.8012
Epoch 6/50
5635/5635 [==============================] - 1s 114us/step - loss: 0.4182 - acc: 0.8043
Epoch 7/50
5635/5635 [==============================] - 1s 128us/step - loss: 0.4165 - acc: 0.8034
Epoch 8/50
5635/5635 [==============================] - 1s 116us/step - loss: 0.4148 - acc: 0.8041
Epoch 9/50
5635/5635 [==============================] - 1s 110us/step - loss: 0.4144 - acc: 0.8067
Epoch 10/50
5635/5635 [==============================] - 1s 108us/step - loss: 0.4129 - acc: 0.8073
Epoch 11/5

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 23.1min finished


Using activation tanh
Epoch 1/25
7043/7043 [==============================] - 12s 2ms/step - loss: 0.5098 - acc: 0.7430
Epoch 2/25
7043/7043 [==============================] - 1s 109us/step - loss: 0.4362 - acc: 0.7947
Epoch 3/25
7043/7043 [==============================] - 1s 107us/step - loss: 0.4266 - acc: 0.8012
Epoch 4/25
7043/7043 [==============================] - 1s 107us/step - loss: 0.4229 - acc: 0.8009
Epoch 5/25
7043/7043 [==============================] - 1s 110us/step - loss: 0.4218 - acc: 0.7999
Epoch 6/25
7043/7043 [==============================] - 1s 111us/step - loss: 0.4202 - acc: 0.8014
Epoch 7/25
7043/7043 [==============================] - 1s 119us/step - loss: 0.4186 - acc: 0.8039
Epoch 8/25
7043/7043 [==============================] - 1s 116us/step - loss: 0.4177 - acc: 0.8026
Epoch 9/25
7043/7043 [==============================] - 1s 112us/step - loss: 0.4159 - acc: 0.8062
Epoch 10/25
7043/7043 [==============================] - 1s 125us/step - loss: 0.4146 - 

**Number of epochs = 25**

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?